# Chapter 16: Surrogate Optimization

In [1]:
import numpy as np
from scipy import stats

## Algorithm 16.1

In [4]:
prob_of_improvement = lambda y_min, mu, sigma: stats.norm(mu,sigma).cdf(y_min)

## Algorithm 16.2

In [7]:
def expected_improvement(y_min, mu, sigma):
    p_imp = prob_of_improvement(y_min, mu, sigma)
    p_ymin = stats.norm(mu, sigma).pdf(y_min)
    return (y_min - mu)*p_imp + sigma*p_ymin

## Algorithm 16.3

In [ ]:
def safe_opt(GP, X, i, f, y_max, beta=3.0, k_max=10):
    GP.X, GP.y = X[i], f(X[i])
    m = len(X)
    u, l = [np.infty for i in range(m)], [-np.infty for i in range(m)]
    S, M, E = [False for i in range(m)], [False for i in range(m)], [False for i in range(m)]

    for k in range(k_max):
        u, l = update_confidence_intervals(GP, X, u, l ,beta)
        S, M, E = compute_sets(GP, S, M, E, X, u, l, y_max, beta)
        i = get_new_query_point(M, E, u, l)
        if i ==0:
            break
        GP.X, GP.y = X[i], f(X[i])

    S = np.less_equal(u, y_max)
    if np.any(S):
        u_best, i_best = np.min(u[S])
        predicate = np.equal(i_best, np.cumsum(S))
        i_best = next(i for i in range(len(i_best)) if predicate[i])
        return u_best, i_best
    else:
        return np.nan, 


## Algorithm 16.4

In [ ]:
def update_confidence_intervals(GP, X, u, l, beta):
    mu_p, vu_p = predict(GP, X)
    u = mu_p + np.sqrt(beta*vu_p)
    l = mu_p - np.sqrt(beta*vu_p)
    return u, l

## Algorithm 16.5

In [ ]:
def compute_sets(GP, S, M, E, X, u, l , y_max, beta):
    M = [False for i in range(len(M))]
    E = [False for i in range(len(E))]

    S = u <= y_max

    if np.any(S):
        M[S] = l[S] < np.minimum(u[S])
        w_max = np.maximum(u[M]-l[M])
        E = S and not M
        if np.any(E):
            E[E] = np.maximum(u[E] - l[E]) > w_max
            for i ,e in enumerate(E):
                if e and u[i] - l[i] > w_max:
                    GP.X, GP.y = X[i], f(X[i])
                    mu_p, vu_p = predict(GP, X)
                    E[i] = np.any(mu_p + np.sqrt(beta*vu_p)>= y_max)
                    if E[i]:
                        w_max = u[i] - l[i]

        return S, M, E
